In [1]:
import requests
from langchain.document_loaders import TextLoader

url = "https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/docs/modules/state_of_the_union.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
    f.write(res.text)

loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()

In [2]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions

client = weaviate.Client(
  embedded_options = EmbeddedOptions()
)

vectorstore = Weaviate.from_documents(
    client = client,    
    documents = chunks,
    embedding = OpenAIEmbeddings(),
    by_text = False
)

/opt/homebrew/Caskroom/miniforge/base/envs/ev/lib/python3.9/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(
{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-04-13T14:13:42-04:00"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-04-13T14:13:42-04:00"}
{"level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-04-13T14:13:42-04:00"}
{"level":"warning"

Started /Users/akashvarun/.cache/weaviate-embedded: process ID 73617


{"action":"restapi_management","level":"info","msg":"Serving weaviate at http://127.0.0.1:8079","time":"2024-04-13T14:13:43-04:00"}
{"level":"info","msg":"Created shard langchain_55f9933fb2674fadbde6b7713d6f035d_B2A0qJP8LTMN in 1.733708ms","time":"2024-04-13T14:13:43-04:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-04-13T14:13:43-04:00","took":55584}
{"action":"read_disk_use","level":"warning","msg":"disk usage currently at 88.35%, threshold set to 80.00%","path":"/Users/akashvarun/.local/share/weaviate","time":"2024-04-13T14:13:43-04:00"}
{"level":"info","msg":"Completed loading shard langchain_7104599c108c4f44b865dcfe0d6aabde_wbuyL68dxCch in 2.806125ms","time":"2024-04-13T14:13:44-04:00"}
{"level":"info","msg":"Completed loading shard langchain_999f7a82c0014e6991ed3bf79692b2c5_nLDRa52sdCEQ in 2.651583ms","time":"2024-04-13T14:13:44-04:00"}
{"level":"info","msg":"Completed load

In [5]:
retriever = vectorstore.as_retriever()


{"action":"read_disk_use","level":"warning","msg":"disk usage currently at 88.35%, threshold set to 80.00%","path":"/Users/akashvarun/.local/share/weaviate","time":"2024-04-13T14:14:14-04:00"}


In [6]:
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don't know the answer, just say that you don't know. \nUse three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:\n"))]


In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

query = "What did the president say about Justice Breyer"
rag_chain.invoke(query)

sys:1: ResourceWarning: unclosed <ssl.SSLSocket fd=83, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('10.0.0.241', 50403), raddr=('104.18.7.192', 443)>


"The president honored Justice Breyer for his service and dedication to the country. He mentioned nominating Judge Ketanji Brown Jackson to continue Justice Breyer's legacy of excellence. The president highlighted Justice Breyer's contributions and the importance of nominating someone to the Supreme Court."

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

query = "What is embedding?"
rag_chain.invoke(query)

sys:1: ResourceWarning: unclosed <ssl.SSLSocket fd=83, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('10.0.0.241', 50418), raddr=('104.18.7.192', 443)>


"I don't know."

{"action":"read_disk_use","level":"warning","msg":"disk usage currently at 88.35%, threshold set to 80.00%","path":"/Users/akashvarun/.local/share/weaviate","time":"2024-04-13T14:16:14-04:00"}
